# Welcome to Core Session 3!

## AGENDA

1. OpenAI Agents SDK  
  - Recap of Async Python and Pydantic
  - Agents with Tools and Structured Outputs 
  - 3 ways for Agents to Collaborate  
2. Use Case: Research Agent
  - Deep Research Prototype
  - Deep Research App
3. Going Deeper 
  - Coder Agents
  - Operator Agents
  - Digital Me revisited


## Approach

- Interactive
- More coding
- Fewer slides...
- Some more advanced stuff

# Lab 1: Foundations

## Topic 1: Async

In [ ]:
# from functions to co-routines

def hello():
    print("Hey!")

# And now call it:


In [ ]:
async def hello():
    print("Hey!")

# And now call it without await:

In [ ]:
async def hello():
    print("Hey!")

# And now call it with await:

In [ ]:
await hello()
await hello()
await hello()

In [2]:
import asyncio

async def do_some_work(id):
    print(f"Starting work {id}")
    await asyncio.sleep(1)
    print(f"Completed work {id}")

In [ ]:
await do_some_work(1)

In [4]:
coroutine = do_some_work(1)

In [ ]:
await coroutine

In [ ]:
await do_some_work(1)
await do_some_work(2)
await do_some_work(3)

In [ ]:
# And now!!

await asyncio.gather(do_some_work(1), do_some_work(2), do_some_work(3))


In [8]:
# But - check this out:

import time

async def do_some_work(id):
    print(f"Starting work {id}")
    time.sleep(1)
    print(f"Completed work {id}")

In [ ]:
await asyncio.gather(do_some_work(1), do_some_work(2), do_some_work(3))

## It looks like threading.. but it's not..

"Cooperative Multi-tasking"

Discuss the similarities and differences

When is it better to use multi-threading (or multi-processing) versus Async?

Why is it so common with LLM projects, and Agents in particular? Why not use multi-processing?

## Topic 2: Pydantic

Pydantic is a Data Validation and Schema framework.

Useful for describing the format of structured data (like json), in a way that allows us to be more rigorous. 

In [10]:
from pydantic import BaseModel, Field
from typing import Optional

In [ ]:
# The old way

book_dict = {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald"}
print(book_dict["title"])
print(book_dict["author"])



In [ ]:
# The new way

class Book(BaseModel):
    title: str
    author: str

book = Book(title="The Great Gatsby", author="F. Scott Fitzgerald")
print(book.title)
print(book.author)
print(book)

In [ ]:
# Convenient shortcut, rather than specifying each field

book = Book(**book_dict)
book

In [ ]:
# Newer way in latest Pydantic

book = Book.model_validate(book_dict)
book

### So structured data in Python could be represented in 3 ways:

1. As a JSON string
2. As a Python dict
3. As a Pydantic object

Let's explore all 3, and see how to convert between them

In [ ]:
book_json = book.model_dump_json()

print(book_json)
print(book_dict)
print(book)

In [17]:
this_is_json = '{"title": "The Great Gatsby", "author": "F. Scott Fitzgerald"}'
this_is_dict = {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald"}
this_is_pydantic_object = Book(title="The Great Gatsby", author="F. Scott Fitzgerald")


In [ ]:
book_json = book.model_dump_json(indent=4)
print(book_json)

In [ ]:
# ERROR

book = Book(**book_json)

In [ ]:
book = Book.model_validate_json(book_json)
book

In [ ]:
# To dict

book.model_dump()

In [ ]:
# To json

book.model_dump_json()

## Pydantic Validation - Optional vs Default

This is confusing!

You can set default values and Optional types, per below. But this might not have the exact implications that you expect..

In [26]:
class BookCheck(BaseModel):
    title: str = "Untitled"
    author: Optional[str]


In [ ]:
# Is this OK?

book_json = '{"title": "The Great Gatsby"}'
book = BookCheck.model_validate_json(book_json)

In [ ]:
# Is this OK?

book_json = '{"author": null}'
book = BookCheck.model_validate_json(book_json)
book

## Pydantic Validation - new fields

Silently discards extra fields by default; can be strict, and can accept extras

In [ ]:
book_special_json = {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "pages": 180}
book_special_json

book_special = Book.model_validate(book_special_json)
book_special

In [ ]:
book_special

In [ ]:
class BookStrict(BaseModel):
    title: str
    author: str

    class Config:
        extra = "forbid"

book_strict = BookStrict.model_validate(book_special_json)
book_strict

In [32]:
class BookFlexible(BaseModel):
    title: str
    author: str

    class Config:
        extra = "allow"

In [ ]:
book_special = BookFlexible.model_validate(book_special_json)
book_special

## Annotating fields

You can add descriptions to fields.

In [ ]:
class BookExplained(BaseModel):
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")


BookExplained.model_json_schema()